In [1]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
from datetime import date

# Data Prep

In [2]:
df = pd.read_csv('./Data/world_info.csv')

path = './Data/Countries'
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]

t = pd.date_range(start='1/1/2020', end='27/3/2020')#date.today().strftime("%d/%m/%Y"))
dt = pd.DataFrame(np.arange(0,len(t), 1), index=t, columns=['Day0'])

countries = []
for c in onlyfiles:
    df = pd.read_csv(path+'/'+c)
    df['Dates'] = df['Dates'].astype(str) + ' 2020'
    df['Dates'] = pd.to_datetime(df['Dates'],format='%b %d %Y')
    df.set_index(df['Dates'],inplace=True)
    df = df.merge(dt,how='right',left_index=True, right_index=True,copy=False)
    df.fillna(0,inplace=True)
    df['Country'] = str.split(c,'.')[0]
    df.reset_index(inplace=True, drop=True)
    df = df[['Day0','Country','Dates','Cases','Deaths','Active Cases','Daily Deaths']]
    df['NewCasesLastWeek']=df['Cases'].diff(periods=7)
    df['NewDeathsLastWeek']=df['Deaths'].diff(periods=7)
    df.fillna(0,inplace=True)
    countries.append(df)

df = pd.concat(countries)
df = df.sort_values(by='Day0')
df.reset_index(inplace=True,drop=True)        

# parameters
mdeaths = int(df['Deaths'].max())
mcases = int(df['Cases'].max())
mnewcases = int(df['NewCasesLastWeek'].max())
mnewdeaths = int(df['NewDeathsLastWeek'].max())
#df.to_csv('./Data/Test/timeseries.csv')
df

,Day0,Country,Dates,Cases,Deaths,Active Cases,Daily Deaths,NewCasesLastWeek,NewDeathsLastWeek
0,0,Netherlands,0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,Philippines,0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,India,0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,S,0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,Denmark,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2779,86,Switzerland,2020-03-27 00:00:00,12928.0,231.0,11167.0,39.0,7313.0,175.0
2780,86,Iran,2020-03-27 00:00:00,32332.0,2378.0,18821.0,144.0,12688.0,945.0
2781,86,China,2020-03-27 00:00:00,81394.0,3295.0,3128.0,3.0,386.0,40.0
2782,86,Spain,2020-03-27 00:00:00,65719.0,5138.0,51224.0,773.0,44148.0,4045.0


# Area Chart

In [ ]:
for col in ['Cases', 'Deaths','Active Cases','Daily Deaths','NewCasesLastWeek','NewDeathsLastWeek']:
    
    fig = px.area(df, x="Day0", y=col, color="Country",)
    fig.update_layout(title=col+' by Country over Time')
    fig.show()

# Line Chart

In [ ]:
for col in ['Cases', 'Deaths','Active Cases','Daily Deaths','NewCasesLastWeek','NewDeathsLastWeek']:
    
    fig = px.line(df, x="Day0", y=col, color="Country",)
    fig.update_layout(title=col+' by Country over Time')
    fig.show()

# Animations

In [ ]:
fig = px.scatter(df, x="Cases", y="Deaths", animation_frame="Day0",
           size="Daily Deaths", color="Country", hover_name="Country",animation_group="Country",text="Country",
                 range_x=[0,mcases], range_y=[0,mdeaths],trendline='ols')
fig.update_layout(title='Evolution of Deaths and Cases by Country over Time')
fig.show()

In [ ]:
fig = px.line(df, x="Cases", y="NewCasesLastWeek",
              color="Country", hover_name="Country",
                 range_x=[500,mcases], range_y=[400,mnewcases],log_x=True,log_y=True,)
fig.update_layout(title='Evolution of New Cases Last Week and Cases by Country over Time')
fig.show()

In [ ]:
fig = px.line(df, x="Cases", y="NewDeathsLastWeek",
              color="Country", hover_name="Country",
                 range_x=[500,mcases], range_y=[1,mnewdeaths],log_x=True,log_y=True,)
fig.update_layout(title='Evolution of New Deaths Last Week and Cases by Country over Time')
fig.show()

In [ ]:
fig = px.line(df, x="Cases", y="NewCasesLastWeek",animation_frame='Day0',
              color="Country", hover_name="Country",text="Country",
                 range_x=[500,mcases], range_y=[400,mnewcases],log_x=True,log_y=True,)
fig.update_layout(title='Evolution of New Cases Last Week and Cases by Country over Time')
fig.show()


In [ ]:
fig = px.line(df, x="Cases", y="NewDeathsLastWeek",animation_frame='Day0',
              color="Country", hover_name="Country",text="Country",
                 range_x=[500,mcases], range_y=[1,mnewdeaths],log_x=True,log_y=True,)
fig.update_layout(title='Evolution of New Deaths Last Week and Cases by Country over Time')
fig.show()



# Leathality

In [24]:
x=np.arange(10,mcases*10,10000)
y=np.arange(10,mdeaths*10,10000)
lethality = [0.005,0.01,0.02,0.04,0.06,0.1]
greyscale = np.flip(np.cumsum([250/len(lethality) for x in lethality]))
lgs = list(zip(lethality,greyscale))
countries = ['China','USA','Germany','Italy','UK','Turkey','Spain','France']


fig = go.Figure()

for l,gs in lgs:
    fig.add_trace(go.Scatter(x=x, y=x*l, mode='lines',
        name=str(round(l*100,1))+'%',text=str(round(l*100,0))+'%',
        marker_color='rgb({},{},{})'.format(gs,gs,gs)
                            )
                 )


for c in countries:
    fig.add_trace(go.Scatter(x=df[df['Country']==c]['Cases'],
                             y=df[df['Country']==c]['Deaths'],
                             mode='lines+markers',text=c,name=c))

fig.update_layout(title="Cases, Deaths and Lethality Iso Lines",
                  xaxis_title="Cases",
                  yaxis_title="Deaths",
                  xaxis_type="log", yaxis_type="log",)
fig.update_xaxes(range=[1, 6])
fig.update_yaxes(range=[0.4, 4.2])  
fig.show()